In [9]:
# pip install opencv-python mediapipe numpy pillow
# pip install torch torchvision # For deep learning tasks, if needed
#

In [ ]:
""" 
Face and Landmark Detection

Use MediaPipe Face Mesh to detect the face and facial landmarks 

"""

import cv2
import mediapipe as mp
import numpy as np
from PIL import Image

# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)


In [7]:
""" Overlay Sunglasses Using OpenCV """

def overlay_sunglasses(frame, sunglasses, landmarks):
    # Compute the position and size of the sunglasses
    left_eye = landmarks[33]
    right_eye = landmarks[263]
    width = int(abs(right_eye[0] - left_eye[0]) * 2)
    height = int(width * sunglasses.size[1] / sunglasses.size[0])
    x = int((left_eye[0] + right_eye[0]) / 2 - width / 2)
    y = int((left_eye[1] + right_eye[1]) / 2 - height / 2)

    # Clamp the coordinates to frame bounds
    x = max(0, x)
    y = max(0, y)
    width = min(width, frame.shape[1] - x)
    height = min(height, frame.shape[0] - y)

    if width > 0 and height > 0:
        sunglasses_resized = sunglasses.resize((width, height), Image.Resampling.LANCZOS)

        # Show resized sunglasses for debugging
        sunglasses_resized.show()  # This will show the resized sunglasses image


        # Convert frame to RGBA for alpha composition
        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).convert("RGBA")

        # Overlay sunglasses
        #frame_pil.alpha_composite(
        #    Image.new("RGBA", frame_pil.size),
        #    (x, y),
        #    sunglasses_resized
        #)
        frame_pil.paste(sunglasses_resized, (x, y), sunglasses_resized)

        # Convert back to BGR
        return cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)
    else:
        print("Sunglasses placement out of bounds.")
        return frame
    
# Load sunglasses
sunglasses = Image.open("sunglasses.png").convert("RGBA")



In [8]:
# Face Identification
# Webcam feed
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    # Draw landmarks
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for id, landmark in enumerate(face_landmarks.landmark):
                h, w, c = frame.shape
                x, y = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(frame, (x, y), 2, (255, 0, 0), -1)

    cv2.imshow("Sunglasses Try-On", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release() # This stops the connection to the camera hardware.
cv2.destroyAllWindows() # This ensures any OpenCV-created GUI windows are closed.

# In the webcam loop:
if results.multi_face_landmarks:
    landmarks = [
        (int(lm.x * w), int(lm.y * h))
        for lm in results.multi_face_landmarks[0].landmark
    ]
    frame = overlay_sunglasses(frame, sunglasses, landmarks)
cv2.imshow("Sunglasses Try-On", frame)

In [ ]:
""" Check image for transparency 

from PIL import Image
import numpy as np

# Load the sunglasses image (with transparency)
sunglasses = Image.open("sunglasses.png").convert("RGBA")

# Resize sunglasses (example resizing)
sunglasses_resized = sunglasses.resize((200, 100), Image.Resampling.LANCZOS)

# Check if the image has transparency
print("Image mode:", sunglasses_resized.mode)  # Should print 'RGBA'

# Get the alpha channel (transparency)
alpha_channel = sunglasses_resized.split()[3]  # 'A' channel is the alpha channel
alpha_array = np.array(alpha_channel)

# Check for transparency (alpha == 0 means transparent)
if np.any(alpha_array == 0):
    print("The image has transparent regions.")
else:
    print("The image is fully opaque.")

# Show the image
sunglasses_resized.show()

"""

In [ ]:
import cv2
import mediapipe as mp
from PIL import Image
import numpy as np

# Function to overlay sunglasses
def overlay_sunglasses(frame, sunglasses, landmarks):
    left_eye = landmarks[33]
    right_eye = landmarks[263]
    width = int(abs(right_eye[0] - left_eye[0]) * 2)
    height = int(width * sunglasses.size[1] / sunglasses.size[0])
    x = int((left_eye[0] + right_eye[0]) / 2 - width / 2)
    y = int((left_eye[1] + right_eye[1]) / 2 - height / 2)

    # Clamp to frame bounds
    x = max(0, x)
    y = max(0, y)
    width = min(width, frame.shape[1] - x)
    height = min(height, frame.shape[0] - y)

    if width > 0 and height > 0:
        sunglasses_resized = sunglasses.resize((width, height), Image.Resampling.LANCZOS)
        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).convert("RGBA")
        frame_pil.paste(sunglasses_resized, (x, y), sunglasses_resized)
        return cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)

    print("Sunglasses placement out of bounds.")
    return frame

# Load sunglasses image
sunglasses = Image.open("sunglasses.png").convert("RGBA")

# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# Start webcam feed
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    # Convert frame to RGB for Mediapipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    # Process face landmarks
    if results.multi_face_landmarks:
        h, w, c = frame.shape
        landmarks = [
            (int(lm.x * w), int(lm.y * h))
            for lm in results.multi_face_landmarks[0].landmark
        ]
        frame = overlay_sunglasses(frame, sunglasses, landmarks)

    # Display the frame
    cv2.imshow("Sunglasses Try-On", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
